In [1]:
import numpy as np
import tensorflow as tf
import keras
import time
from keras import layers
from tensorflow.keras.applications import EfficientNetB0

2024-03-17 07:40:48.976842: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Load data

In [2]:
(datain_tr, dataout_tr), (datain_te, dataout_te) = tf.keras.datasets.cifar10.load_data()

dataout_tr = keras.utils.to_categorical(dataout_tr)
dataout_te = keras.utils.to_categorical(dataout_te)

print('Shape of datain_tr: {}'.format(datain_tr.shape))
print('Shape of datain_te: {}'.format(datain_te.shape))
print('Shape of dataou_tr: {}'.format(dataout_tr.shape))
print('Shape of dataou_te: {}'.format(dataout_te.shape))

CIFAR_IMG_SHAPE = (32, 32, 3)
IMG_SIZE = 224
BATCH_SIZE = 64


Shape of datain_tr: (50000, 32, 32, 3)
Shape of datain_te: (10000, 32, 32, 3)
Shape of dataou_tr: (50000, 10)
Shape of dataou_te: (10000, 10)


## Define some input and output layers

In [3]:
# Input layer
layerin = layers.Input(
    shape=CIFAR_IMG_SHAPE
)

# Upscaling layer (we can also upscale the images before)
# EfficientNetB0 use 224x224x3 images
upscale = layers.Lambda(
    lambda x: tf.image.resize_with_pad(
        x,
        IMG_SIZE,
        IMG_SIZE,
        method=tf.image.ResizeMethod.BILINEAR
    )
)(layerin)

# Output layer. Dense with 10 classes
layerout = layers.Dense(dataout_tr.shape[-1], activation='softmax')


## Fine-Tuning without Transfer Learning

In [4]:
# Model definition
model_base = EfficientNetB0(
    include_top=False,
    weights=None,
    input_shape=CIFAR_IMG_SHAPE,
    input_tensor = upscale,
    pooling='max'
)

model = keras.models.Sequential([
    model_base,
    layerout
])

2024-03-17 07:40:55.674384: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4453 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1660, pci bus id: 0000:29:00.0, compute capability: 7.5


In [5]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=["accuracy"]
)
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ ?                      │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,049,571 (15.45 MB)

 Trainable params: 4,007,548 (15.29 MB)

 Non-trainable params: 42,023 (164.16 KB)

In [ ]:
t_start = time.time()

history = model.fit(datain_tr, dataout_tr, epochs = 5, batch_size = BATCH_SIZE,
                    verbose = 1, shuffle = True, validation_split = 0.05)

t_end   = time.time()

t_fine_tuning = t_end - t_start

Epoch 1/5
743/743 [==============================] - 408s 496ms/step - loss: 2.0244 - accuracy: 0.3290 - val_loss: 5.4570 - val_accuracy: 0.3756
Epoch 2/5
743/743 [==============================] - 366s 492ms/step - loss: 1.3072 - accuracy: 0.5536 - val_loss: 1.1229 - val_accuracy: 0.6292
Epoch 3/5
743/743 [==============================] - 365s 491ms/step - loss: 1.0071 - accuracy: 0.6672 - val_loss: 1.0771 - val_accuracy: 0.6684
Epoch 4/5
743/743 [==============================] - 366s 492ms/step - loss: 0.8419 - accuracy: 0.7263 - val_loss: 1.5352 - val_accuracy: 0.7420
Epoch 5/5
743/743 [==============================] - 366s 492ms/step - loss: 0.6888 - accuracy: 0.7816 - val_loss: 0.6621 - val_accuracy: 0.7880

Fine tuning training time: 1901.44 sec


In [ ]:
_, accuracy_te = model.evaluate(datain_te,
                                dataout_te,
                                batch_size = 128)

print('\nTotal Training time: {:06.2f} sec'.format(t_fine_tuning))
print('\nTesting accuracy: {:05.2f}%'.format(accuracy_te * 100))

## Transfer Learning with Fine-Tuning and Linear Probing

In [6]:
# Model definition
model_base = EfficientNetB0(
    include_top=False,
    weights='imagenet',
    input_shape=CIFAR_IMG_SHAPE,
    input_tensor = upscale,
    pooling='max'
)

model = keras.models.Sequential([
    model_base,
    layerout
])

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=["accuracy"]
)

model_base.trainable = False

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=["accuracy"]
)

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ ?                      │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,049,571 (15.45 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 4,049,571 (15.45 MB)

In [7]:
t_start = time.time()

history = model.fit(datain_tr, dataout_tr, epochs = 3, batch_size = BATCH_SIZE,
                    verbose = 1, shuffle = True, validation_split = 0.05)

t_end   = time.time()

t_transfer_learning = t_end - t_start

Epoch 1/3


I0000 00:00:1710661311.961490   15693 service.cc:145] XLA service 0x7f7314001df0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1710661311.961523   15693 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce GTX 1660, Compute Capability 7.5
2024-03-17 07:41:52.250261: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
W0000 00:00:1710661312.345938   15693 assert_op.cc:38] Ignoring Assert operator sequential_1_1/efficientnetb0_1/lambda_1/resize_image_with_pad/pad_to_bounding_box/Assert/Assert
W0000 00:00:1710661312.346117   15693 assert_op.cc:38] Ignoring Assert operator sequential_1_1/efficientnetb0_1/lambda_1/resize_image_with_pad/pad_to_bounding_box/Assert_1/Assert
W0000 00:00:1710661312.346199   15693 assert_op.cc:38] Ignoring Assert operator sequential_1_1/efficientnetb0_1/lambda_1/resize_image_with_pad/pad_to_bounding_

  2/743 ━━━━━━━━━━━━━━━━━━━━ 57s 78ms/step - accuracy: 0.1172 - loss: 5.6806   

I0000 00:00:1710661332.848092   15693 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


742/743 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.6385 - loss: 1.3471

W0000 00:00:1710661386.207062   15690 assert_op.cc:38] Ignoring Assert operator sequential_1_1/efficientnetb0_1/lambda_1/resize_image_with_pad/pad_to_bounding_box/Assert/Assert
W0000 00:00:1710661386.207103   15690 assert_op.cc:38] Ignoring Assert operator sequential_1_1/efficientnetb0_1/lambda_1/resize_image_with_pad/pad_to_bounding_box/Assert_1/Assert
W0000 00:00:1710661386.207115   15690 assert_op.cc:38] Ignoring Assert operator sequential_1_1/efficientnetb0_1/lambda_1/resize_image_with_pad/pad_to_bounding_box/Assert_2/Assert
W0000 00:00:1710661386.207127   15690 assert_op.cc:38] Ignoring Assert operator sequential_1_1/efficientnetb0_1/lambda_1/resize_image_with_pad/pad_to_bounding_box/Assert_3/Assert


743/743 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - accuracy: 0.6386 - loss: 1.3465

W0000 00:00:1710661402.076733   15690 assert_op.cc:38] Ignoring Assert operator sequential_1_1/efficientnetb0_1/lambda_1/resize_image_with_pad/pad_to_bounding_box/Assert/Assert
W0000 00:00:1710661402.077083   15690 assert_op.cc:38] Ignoring Assert operator sequential_1_1/efficientnetb0_1/lambda_1/resize_image_with_pad/pad_to_bounding_box/Assert_1/Assert
W0000 00:00:1710661402.077312   15690 assert_op.cc:38] Ignoring Assert operator sequential_1_1/efficientnetb0_1/lambda_1/resize_image_with_pad/pad_to_bounding_box/Assert_2/Assert
W0000 00:00:1710661402.077446   15690 assert_op.cc:38] Ignoring Assert operator sequential_1_1/efficientnetb0_1/lambda_1/resize_image_with_pad/pad_to_bounding_box/Assert_3/Assert
W0000 00:00:1710661406.850340   15693 assert_op.cc:38] Ignoring Assert operator sequential_1_1/efficientnetb0_1/lambda_1/resize_image_with_pad/pad_to_bounding_box/Assert/Assert
W0000 00:00:1710661406.850419   15693 assert_op.cc:38] Ignoring Assert operator sequential_1_1/efficientnetb0

743/743 ━━━━━━━━━━━━━━━━━━━━ 104s 105ms/step - accuracy: 0.6387 - loss: 1.3459 - val_accuracy: 0.8296 - val_loss: 0.5561
Epoch 2/3
743/743 ━━━━━━━━━━━━━━━━━━━━ 56s 76ms/step - accuracy: 0.8100 - loss: 0.5947 - val_accuracy: 0.8412 - val_loss: 0.4922
Epoch 3/3
743/743 ━━━━━━━━━━━━━━━━━━━━ 56s 76ms/step - accuracy: 0.8326 - loss: 0.5193 - val_accuracy: 0.8620 - val_loss: 0.4267


In [8]:
model_base.trainable = True

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=["accuracy"]
)

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 1280)           │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │        12,810 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,062,381 (15.50 MB)

 Trainable params: 4,020,358 (15.34 MB)

 Non-trainable params: 42,023 (164.16 KB)

In [9]:
t_start = time.time()

history = model.fit(datain_tr, dataout_tr, epochs = 2, batch_size = BATCH_SIZE,
                    verbose = 1, shuffle = True, validation_split = 0.05)

t_end   = time.time()

t_fine_tuning = t_end - t_start

print('\nFine-tuning training time: {:06.2f} sec'.format(t_fine_tuning))

Epoch 1/2


W0000 00:00:1710661545.146961   15691 assert_op.cc:38] Ignoring Assert operator sequential_1_1/efficientnetb0_1/lambda_1/resize_image_with_pad/pad_to_bounding_box/Assert/Assert
W0000 00:00:1710661545.147149   15691 assert_op.cc:38] Ignoring Assert operator sequential_1_1/efficientnetb0_1/lambda_1/resize_image_with_pad/pad_to_bounding_box/Assert_1/Assert
W0000 00:00:1710661545.147420   15691 assert_op.cc:38] Ignoring Assert operator sequential_1_1/efficientnetb0_1/lambda_1/resize_image_with_pad/pad_to_bounding_box/Assert_2/Assert
W0000 00:00:1710661545.147493   15691 assert_op.cc:38] Ignoring Assert operator sequential_1_1/efficientnetb0_1/lambda_1/resize_image_with_pad/pad_to_bounding_box/Assert_3/Assert
2024-03-17 07:46:00.658701: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng3{k11=2} for conv (f32[64,96,113,113]{3,2,1,0}, u8[0]{0}) custom-call(f32[64,96,115,115]{3,2,1,0}, f32[96,1,3,3]{3,2,1,0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, featur

743/743 ━━━━━━━━━━━━━━━━━━━━ 329s 356ms/step - accuracy: 0.8245 - loss: 0.7252 - val_accuracy: 0.9284 - val_loss: 0.2159
Epoch 2/2
743/743 ━━━━━━━━━━━━━━━━━━━━ 226s 304ms/step - accuracy: 0.9414 - loss: 0.1765 - val_accuracy: 0.9316 - val_loss: 0.2317

Fine-tuning training time: 554.78 sec


In [10]:
_, accuracy_te = model.evaluate(datain_te,
                                dataout_te,
                                batch_size = BATCH_SIZE)

print('\nTotal Training time: {:06.2f} sec'.format(t_transfer_learning + t_fine_tuning))
print('\nTesting accuracy: {:05.2f}%'.format(accuracy_te * 100))

157/157 ━━━━━━━━━━━━━━━━━━━━ 16s 104ms/step - accuracy: 0.9235 - loss: 0.2375

Total Training time: 772.58 sec

Testing accuracy: 92.71%
